--------------------------------------------------------------------------------


11.03.2022

<b>GENERATIVE FILM-MAKING NOTEBOOK 002</b>

*** VIDEOGREP-ING USING MOVIE.PY ONLY ****

https://zulko.github.io/blog/2014/06/21/some-more-videogreping-with-python/

This notebook shows :

- how to download films from Youtube and automatically cut according to text input

- including forming new sentences from cuts

To do :

- some sort of automatic cuts using most common words could be interesting
- ability to take in multiple films and form cuts across films would be good 
.... perhaps could skip any words which aren´t in original cuts
.... or use enough film content that most words will be there
.... also should automate to not use the same clip twice where possible
.... interactive GPT2 version would be nice also!



In [44]:

# TASK BEFORE STARTING 
# WHICH MIGHT BE NECESSARY FOR SOME

## This is the fix for ImageMagick bug from https://github.com/Zulko/moviepy/issues/347
## ( only needs to be run once on system so just use if needed )

 
# You can find relevant path for Imagemagick on Terminal using "brew info imageMagick"

#from moviepy.config import change_settings
#change_settings({"IMAGEMAGICK_BINARY": "/opt/homebrew/Cellar/imagemagick/7.1.0-27/bin/convert"})





-------------------------------------------------------------------------------------

<b>IMPORT LIBRARIES</b>

In [93]:
import re # module for regular expressions
import os
import sys
import shutil
import glob
import random
import fnmatch
from pathlib import Path

import numpy as np

from moviepy import *
from moviepy.video import *

from moviepy.audio.tools.cuts import find_audio_period
from moviepy.video.tools.cuts import find_video_period

from moviepy.audio.AudioClip import *
from moviepy.editor import VideoFileClip, AudioFileClip, TextClip, concatenate, concatenate_videoclips, clips_array, vfx, CompositeVideoClip



-------------------------------------------------------------------------------------

<b> DOWNLOAD THE CLIPS </b>

In [25]:
# # search topic on Youtube
# topic = "80s godzilla"

# # directory to save into 
# directory = "godzilla"

# # number of films to pull from on Youtube
# numberOfFilms = "3"

# # starting time to grab from
# start = "00:00:30.00"

# # length of segment to grab
# length = "00:00:10.00"


In [171]:
# GENERAL DOWNLOAD OPTIONS
# (download any of first N youtube vids with robots in title and CC license - add them to directory robots)

##### Search top 100 videos by term "80s robots" filter for those with max views 50
#os.system("youtube-dl 'ytsearch100:80s robots' --max-views 50 --output 'robots/%(title)s.mp4' ")

##### filter by --max-filesize (not checked)
#os.system("youtube-dl 'ytsearch100:80s robots' --max-filesize 500 --output 'robots/%(title)s.mp4' ")

# filter by CC license
#os.system("youtube-dl 'ytsearch3:80s robots' --match-filter 'license=Creative Commons' --output 'robots/%(title)s.mp4' ")

########################################################################################
########################################################################################

# DOWNLOAD METHOD FOR SHORT CLIPS ONLY

# t is duration to is end time, ss is start time
#os.system("ffmpeg $(youtube-dl -g 'https://www.youtube.com/watch?v=oHg5SJYRHA0' | sed 's/^/-ss 00:55 -t 01:00 -i /') -c copy out2.mkv")

#os.system("youtube-dl -k --external-downloader ffmpeg --external-downloader-args '-ss 00:00:05.00 -to 00:00:10.00' oHg5SJYRHA0")

#os.system("youtube-dl --autonumber-start 1 -k --external-downloader ffmpeg --external-downloader-args '-ss "+start+" -t "+length+"' 'ytsearch"+numberOfFilms+":"+topic+"' --output '"+directory+"/%(autonumber)s.mp4' ")


# this download command didn´t work for me, needs updating...
# I had to rename the actual file it downloaded and convert the vtt to srt manually
#os.system("youtube-dl --write-srt --srt-lang en Zgfi7wnGZlE state.mp4 ")

# this might fix the output name at least, not sure about the srt format issue
os.system("youtube-dl --write-srt --srt-lang en Zgfi7wnGZlE -o state.mp4 ")



# Note - if gets locked on a single failing dload, stop button flips to next one 

[youtube] Zgfi7wnGZlE: Downloading webpage
[info] Writing video subtitles to: 2012 State Of The Union Address - Enhanced Version-Zgfi7wnGZlE.en.vtt
[download] Resuming download at byte 15617453
[download] Destination: 2012 State Of The Union Address - Enhanced Version-Zgfi7wnGZlE.f136.mp4
[download]  56.5% of 449.86MiB at 54.79KiB/s ETA 01:00:54[download] Got server HTTP error: [Errno 54] Connection reset by peer. Retrying (attempt 1 of 10)...
[download] Destination: 2012 State Of The Union Address - Enhanced Version-Zgfi7wnGZlE.f136.mp4
[download]  57.1% of 449.86MiB at 55.80KiB/s ETA 59:01:03[download] Got server HTTP error: [Errno 54] Connection reset by peer. Retrying (attempt 2 of 10)...
[download] Destination: 2012 State Of The Union Address - Enhanced Version-Zgfi7wnGZlE.f136.mp4
[download]  62.7% of 449.86MiB at 26.07KiB/s ETA 01:49:53[download] Got server HTTP error: [Errno 54] Connection reset by peer. Retrying (attempt 3 of 10)...
[download] Destination: 2012 State Of The Un

ERROR: 'state.mp4' is not a valid URL. Set --default-search "ytsearch" (or run  youtube-dl "ytsearch:state.mp4" ) to search YouTube


256

-------------------------------------------------------------------------------------

<b>1</b>



In [172]:
#### Converts a string into seconds

def convert_time(timestring):
    
    nums = list(map(float, re.findall(r'\d+', timestring)))
    return 3600*nums[0] + 60*nums[1] + nums[2] + nums[3]/1000



In [176]:

### Take all lines in file, return as a list 
### where each line is an item :

with open("state.srt") as f:
    lines = f.readlines()

    
# The times are stored on every 4th line
#print(lines[1])
#print(lines[5])
#lines

In [177]:

# setup some variables to use
times_texts = []
current_times = None
current_text = ""



# iterating over the lines to split out the time data
for line in lines:
    
    #print(line)
    # check for numerical entries and store in "times"
    times = re.findall("[0-9]*:[0-9]*:[0-9]*,[0-9]*", line)
    
    # if numerical entries were found, convert and store in "current_times"
    if times != []:
        current_times = list(map(convert_time, times))
        #print("1")
        #print("times=",times)
    
    # when reaching a new line break, add "current times" and "current text" into array then clear ready for next
    elif line == '\n':
        times_texts.append((current_times, current_text))
        #print("current_times, current_text =", current_times, current_text)
        current_times, current_text = None, ""
        #print("2")
        
    # if not a new line break (and not a line with numerical data), store text and cut out line break
    elif current_times is not None:
        current_text = current_text + line.replace("\n"," ")
        #print("3")
        

#print (times_texts)

In [178]:

# If you don´t edit the other cells according to the most common words then I think it can throw up an error if word doesn´t exist
# Be good to pass the common words automatically... and / or raise an error which explains

from collections import Counter
whole_text = " ".join([text for (time, text) in times_texts])
all_words = re.findall("\w+", whole_text)
counter = Counter([w.lower() for w in all_words if len(w)>5])
print (counter.most_common(500))

[('applause', 82), ('american', 35), ('america', 33), ('because', 25), ('should', 24), ('people', 23), ('energy', 23), ('americans', 20), ('country', 18), ('congress', 15), ('cheering', 15), ('businesses', 13), ('tonight', 12), ('economy', 12), ('workers', 12), ('million', 11), ('business', 11), ('companies', 11), ('states', 10), ('financial', 10), ('nation', 9), ('industry', 9), ('together', 8), ('manufacturing', 8), ('before', 8), ('college', 8), ('millions', 8), ('nearly', 8), ('making', 8), ('education', 8), ('government', 8), ('mission', 7), ('future', 7), ('chance', 7), ('products', 7), ('enough', 7), ('values', 7), ('afford', 7), ('deficit', 7), ('getting', 7), ('company', 7), ('against', 7), ('working', 7), ('through', 7), ('support', 7), ('president', 6), ('united', 6), ('responsibility', 6), ('republican', 6), ('change', 6), ('community', 6), ('administration', 6), ('better', 6), ('innovation', 6), ('federal', 6), ('troops', 5), ('citizens', 5), ('generation', 5), ('control',

In [182]:

# find all cuts involving the word people
cuts = [times for (times,text) in times_texts
        if (re.findall("should",text) != [])]

In [183]:
# A function to concatenate the cuts...kind of overkill / abstracts things
# but working now - needed audio specs in the write videofile command

video = VideoFileClip("state.mp4")

def assemble_cuts(cuts, outputfile):
    """ Concatenate cuts and generate a video file. """
    final = concatenate_videoclips([video.subclip(start, end)
                         for (start,end) in cuts])
    final.write_videofile(outputfile, audio=True, audio_bitrate="1000k", codec='libx264', audio_codec='aac')



In [184]:

assemble_cuts(cuts, "should.mp4")

Moviepy - Building video should.mp4.
MoviePy - Writing audio in shouldTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video should.mp4



Moviepy - Done !
Moviepy - video ready should.mp4


In [185]:
times, texts = zip(*times_texts)
txt_lengths = list(map(len, texts)) # length of each subtitle block
indices = [sum(txt_lengths[:i]) for i in range(len(texts))]

def find_times(position):
    """ Finds the (t_start, t_end) in the subtitles
        for a given position in the whole text. """
    return times[ max([i for i in range(len(indices))
                       if (indices[i] <= position)])]



In [186]:
# # Regular expression matching all sentences with 'should'
# regexpr = "([A-Z][^\.!?]*%s[^\.!?]*[\.!?])"%("people")

# for m in re.finditer(regexpr, whole_text):
#     print(m)

    
# print("wtf")
# print(m)    
# find_times(m.start())[0]

In [189]:
# Regular expression matching all sentences with 'should'
regexpr = "([A-Z][^\.!?]*%s[^\.!?]*[\.!?])"%("should")

cuts = [ (find_times(m.start())[0], find_times(m.end())[1])
         for m in re.finditer(regexpr, whole_text) ]

cuts

[(563.4, 572.567),
 (727.8, 734.133),
 (757.233, 770.0),
 (765.233, 782.767),
 (779.367, 791.867),
 (788.033, 811.233),
 (808.2, 818.467),
 (818.467, 825.1),
 (820.934, 831.333),
 (829.5, 845.166),
 (976.767, 1006.967),
 (1430.467, 1440.6),
 (1491.5, 1499.367),
 (1509.667, 1531.0),
 (1575.3, 1582.4),
 (1582.4, 1597.4),
 (1982.166, 2005.967),
 (2076.467, 2088.533),
 (2185.9, 2213.133),
 (2387.9, 2405.867),
 (2452.834, 2471.633),
 (2625.7, 2634.266),
 (2781.8, 2790.967),
 (2786.266, 2796.5),
 (2793.767, 2804.1),
 (2800.734, 2812.6),
 (2807.633, 2819.5),
 (3080.6, 3089.033),
 (3160.8, 3176.633),
 (3227.033, 3236.567)]

In [190]:
# NOTE : creates a 45 minute file on covid briefing video... 
# also doesn´t seem to work on original Obama video
assemble_cuts(cuts, "should_sentence.mp4")

t:  30%|███████▌                 | 3223/10690 [01:23<02:40, 46.66it/s, now=None]

Moviepy - Building video should_sentence.mp4.
MoviePy - Writing audio in should_sentenceTEMP_MPY_wvf_snd.mp4



chunk:  29%|██████               | 2599/9081 [00:10<00:24, 264.86it/s, now=None]


chunk:  60%|████████████▋        | 5466/9081 [00:21<00:13, 271.30it/s, now=None]


chunk:  93%|███████████████████▍ | 8426/9081 [00:32<00:02, 262.14it/s, now=None]


...

t:  89%|█████████████████████▎  | 10971/12342 [04:40<00:35, 38.73it/s, now=None]


t:  94%|██████████████████████▋ | 11640/12342 [04:56<00:13, 51.42it/s, now=None]


t: 100%|███████████████████████▉| 12332/12342 [05:11<00:00, 48.59it/s, now=None]


t:  30%|███████▌                 | 3223/10690 [07:11<02:40, 46.66it/s, now=None]

Moviepy - Done !
Moviepy - video ready should_sentence.mp4


Greping single words

In [195]:
def find_word(word, padding=.05):
    """ Finds all 'exact' (t_start, t_end) for a word """
    matches = [re.search(word, text)
               for (t,text) in times_texts]
    return [(t1 + m.start()*(t2-t1)/len(text) - padding,
             t1 + m.end()*(t2-t1)/len(text) + padding)
             for m,((t1,t2),text) in zip(matches, times_texts)
             if (m is not None)]

In [192]:
# The “ValueError: max() arg is an empty sequence” error is 
#     raised when you try to find the largest item in an empty list
#     using the max() method. To solve this error, make sure you 
#     only pass lists with at least one value through a max() statement


assemble_cuts( find_word("Americans"), "Americans.mp4")

t:  30%|███████▌                 | 3223/10690 [07:16<02:40, 46.66it/s, now=None]

Moviepy - Building video Americans.mp4.
MoviePy - Writing audio in AmericansTEMP_MPY_wvf_snd.mp4



t:  30%|███████▌                 | 3223/10690 [07:19<02:40, 46.66it/s, now=None]

MoviePy - Done.
Moviepy - Writing video Americans.mp4




t:  30%|███████▌                 | 3223/10690 [07:36<02:40, 46.66it/s, now=None]

Moviepy - Done !
Moviepy - video ready Americans.mp4


In [193]:
# words here must exist for this to run!
# check against the common words print out

# also need to find clearly pronounced words for it to work
# and/or change the padding in the function above

words = ["Americans", "must", "develop", "open ", "source",
          " software", "for the", " rest ", "of the world",
          "instead of", " soldiers"]

numbers = [3,0,4,3,4,0,1,2,0,1,0] # take clip number 'n'

# change this if you want it to select something other than 1st clip 
# ...but be aware will kick up an error if the clip doesn´t exist
# be good to fix

# safer to use lower numbers though in general e.g.
# numbers = [0,1,2,3,4,0,0,0,1,2,3] 


cuts = [find_word(word)[n] for (word,n) in zip(words, numbers)]
assemble_cuts(cuts, "obama_speech.mp4")

t:  30%|███████▌                 | 3223/10690 [07:39<02:40, 46.66it/s, now=None]

Moviepy - Building video obama_speech.mp4.
MoviePy - Writing audio in obama_speechTEMP_MPY_wvf_snd.mp4



t:  30%|███████▌                 | 3223/10690 [07:41<02:40, 46.66it/s, now=None]

MoviePy - Done.
Moviepy - Writing video obama_speech.mp4




t:  30%|███████▌                 | 3223/10690 [07:49<02:40, 46.66it/s, now=None]

Moviepy - Done !
Moviepy - video ready obama_speech.mp4


In [194]:
cuts = [times for (times,text) in times_texts
              if (re.findall("applause",text) != [])]
assemble_cuts(cuts, "applause_obama.mp4")

t:  30%|███████▌                 | 3223/10690 [08:09<02:40, 46.66it/s, now=None]

Moviepy - Building video applause_obama.mp4.
MoviePy - Writing audio in applause_obamaTEMP_MPY_wvf_snd.mp4



chunk:  16%|███▏                | 3101/19340 [00:10<00:50, 320.88it/s, now=None]


chunk:  31%|██████▏             | 5931/19340 [00:21<00:51, 260.59it/s, now=None]


chunk:  47%|█████████▎          | 9059/19340 [00:32<00:36, 280.53it/s, now=None]


...

t:  94%|██████████████████████▌ | 24651/26286 [09:32<00:37, 43.83it/s, now=None]


t:  96%|███████████████████████▏| 25355/26286 [09:49<00:16, 57.37it/s, now=None]


t:  99%|███████████████████████▊| 26074/26286 [10:04<00:06, 34.42it/s, now=None]


t:  30%|███████▌                 | 3223/10690 [19:29<02:40, 46.66it/s, now=None]

Moviepy - Done !
Moviepy - video ready applause_obama.mp4


In [39]:

 
# iterate over the downloaded files in the new directory to remove unwanted fragments

for filename in os.listdir(directory):
    if fnmatch.fnmatch(filename, '*.mp4'):
        print("ok")
    elif fnmatch.fnmatch(filename, '*.mkv'):
        print("ok")
    else :
        f = os.path.join(directory, filename)
        os.remove(f)

# print out what remains

for filename in os.listdir(directory):
    print(filename)
        
        

ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
00012.mp4
00004.mp4
00010.mp4
00005.mp4
00016.mp4
00002.mp4
00003.mp4
00008.mp4
00011.f137.mp4
00014.mp4.mkv


----------------------------------------------------------------------------------------------------

<b>CUT FILES INTO PIECES AND CONCATENATE</b>

In [41]:
########################################################################################
#### Take a random cut of a few seconds from anywhere within the duration of the clip ##
#### Add it to the array and concatenate them altogether for output ####################
########################################################################################

# temp = VideoFileClip("temp.mp4", audio=False) 

# allClips = []

# for i in range(1,20):
#     randE = random.randint(3,int(temp.duration))
#     allClips.append(eider2.subclip(randE,randE+2))

# final_clip = concatenate_videoclips(allClips)
# final_clip.write_videofile("my_concatenation2E.mp4")

In [42]:
########################################################################################
# Automate process across an entire directory
########################################################################################

# NOTE : This is glitchy at the moment, which is nice but... seems to be to do with size of vide
# Need to add a crop or command to deal with different types of videos ideally
########################################################################################
########################################################################################


allClips = []

# iterate over directory
for filename in os.listdir(directory):
    
    print(filename)
    
    # create videoclip
    temp = VideoFileClip(directory+"/"+filename, audio=False) 
    
    # get duration of clip
    tDuration = temp.duration
    
    # check long enough to cut portion out
    if tDuration >5 :
        
        # take 5 second segment from random point in clip 
        #randE = random.randint(1,int(tDuration))
        allClips.append(temp.subclip(5,8))
        
    # if not long enough, add entire clip
    else :
        allClips.append(temp)

        
final_clip = concatenate_videoclips(allClips, method="compose")
final_clip.write_videofile("all_"+topic+".mp4")



00012.mp4
00004.mp4
00010.mp4
00005.mp4
00016.mp4
00002.mp4
00003.mp4
00008.mp4
00011.f137.mp4
00014.mp4.mkv


t:   1%|▎                         | 209/18054 [34:50<39:47,  7.47it/s, now=None]

Moviepy - Building video all_80s godzilla.mp4.
Moviepy - Writing video all_80s godzilla.mp4




t:  11%|██▉                        | 195/1800 [00:29<04:02,  6.61it/s, now=None]


t:  22%|█████▊                     | 389/1800 [00:58<03:39,  6.43it/s, now=None]


...

t:  65%|████████████████▊         | 1165/1800 [03:00<01:24,  7.47it/s, now=None]


t:  79%|████████████████████▌     | 1427/1800 [03:32<00:53,  6.94it/s, now=None]


t: 100%|█████████████████████████▉| 1793/1800 [04:04<00:00, 13.06it/s, now=None]


t:   1%|▎                         | 209/18054 [38:55<39:47,  7.47it/s, now=None]

Moviepy - Done !
Moviepy - video ready all_80s godzilla.mp4


In [43]:
# Need to add in a step to resize / crop clips to same dimensions